# Mosaico 2008

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [4]:
import geemap.colormaps as cm

In [5]:
# cm.palettes.terrain

In [6]:
color_ndvi = cm.palettes.ndvi

In [7]:
color_ndwi = cm.palettes.ndwi

In [8]:
color_terrain = cm.palettes.terrain

In [9]:
color_glaciar = cm.palettes.PuBuGn_r

In [10]:
color_nubes = cm.palettes.inferno

In [11]:
# cm.list_colormaps()

In [12]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [13]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [14]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('B11')
    return image.addBands(ndvi)

In [15]:
def addNDVI7(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [16]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [17]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [18]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [19]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [20]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [21]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [22]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [23]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [24]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2009

In [25]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l5_2009 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [26]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [27]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [28]:
coleccion2009 = l5_2009.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [29]:
QM_NDVI_2009 = coleccion2009.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2009 = coleccion2009.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2009 = coleccion2009.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2009 = coleccion2009.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2009 = coleccion2009.qualityMosaic('NDSI') # Normalized difference snow index

In [30]:
Map.addLayer(QM_NDVI_2009, vizParams, 'QM de NDVI 2009')
Map.addLayer(QM_NDWI_2009, vizParams, 'QM de NDWI 2009')
Map.addLayer(QM_SAVI_2009, vizParams, 'QM de SAVI 2009')
Map.addLayer(QM_NDGI_2009, vis_nieve, 'QM de NDGI 2009')
Map.addLayer(QM_NDSI_2009, vis_nieve, 'QM de NDSI 2009')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [31]:
c2009_sn = coleccion2009.map(Cloud20)
NDVI_2009_sn = c2009_sn.qualityMosaic('B11')
NDWI_2009_sn = c2009_sn.qualityMosaic('NDWI')
SAVI_2009_sn = c2009_sn.qualityMosaic('SAVI')
NDGI_2009_sn = c2009_sn.qualityMosaic('NDGI')
NDSI_2009_sn = c2009_sn.qualityMosaic('NDSI')

In [32]:
# Map2.addLayer(NDVI_2009_sn, vizParams, 'QMsn de NDVI 2009')
# Map2.addLayer(NDWI_2009_sn, vizParams, 'QMsn de NDWI 2009')
# Map2.addLayer(SAVI_2009_sn, vizParams, 'QMsn de SAVI 2009')
# Map2.addLayer(NDGI_2009_sn, vis_nieve, 'QMsn de NDGI 2009')
# Map2.addLayer(NDSI_2009_sn, vis_nieve, 'QMsn de NDSI 2009')
# Map2

### 2008

In [33]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l5_2008 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [34]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [35]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [36]:
coleccion2008 = l5_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [37]:
QM_NDVI_2008 = coleccion2008.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2008 = coleccion2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008 = coleccion2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008 = coleccion2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008 = coleccion2008.qualityMosaic('NDSI') # Normalized difference snow index

In [38]:
Map.addLayer(QM_NDVI_2008, vizParams, 'QM de NDVI 2008')
Map.addLayer(QM_NDWI_2008, vizParams, 'QM de NDWI 2008')
Map.addLayer(QM_SAVI_2008, vizParams, 'QM de SAVI 2008')
Map.addLayer(QM_NDGI_2008, vis_nieve, 'QM de NDGI 2008')
Map.addLayer(QM_NDSI_2008, vis_nieve, 'QM de NDSI 2008')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [39]:
c2008_sn = coleccion2008.map(Cloud20)
NDVI_2008_sn = c2008_sn.qualityMosaic('B11')
NDWI_2008_sn = c2008_sn.qualityMosaic('NDWI')
SAVI_2008_sn = c2008_sn.qualityMosaic('SAVI')
NDGI_2008_sn = c2008_sn.qualityMosaic('NDGI')
NDSI_2008_sn = c2008_sn.qualityMosaic('NDSI')

In [40]:
# Map2.addLayer(NDVI_2008_sn, vizParams, 'QMsn de NDVI 2008')
# Map2.addLayer(NDWI_2008_sn, vizParams, 'QMsn de NDWI 2008')
# Map2.addLayer(SAVI_2008_sn, vizParams, 'QMsn de SAVI 2008')
# Map2.addLayer(NDGI_2008_sn, vis_nieve, 'QMsn de NDGI 2008')
# Map2.addLayer(NDSI_2008_sn, vis_nieve, 'QMsn de NDSI 2008')
# Map2

### 2007

In [41]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l5_2007 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [42]:
coleccion2007 = l5_2007.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [43]:
QM_NDVI_2007 = coleccion2007.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2007 = coleccion2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007 = coleccion2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007 = coleccion2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007 = coleccion2007.qualityMosaic('NDSI') # Normalized difference snow index

In [44]:
Map.addLayer(QM_NDVI_2007, vizParams, 'QM de NDVI 2007')
Map.addLayer(QM_NDWI_2007, vizParams, 'QM de NDWI 2007')
Map.addLayer(QM_SAVI_2007, vizParams, 'QM de SAVI 2007')
Map.addLayer(QM_NDGI_2007, vis_nieve, 'QM de NDGI 2007')
Map.addLayer(QM_NDSI_2007, vis_nieve, 'QM de NDSI 2007')
# Map3

In [45]:
c2007_sn = coleccion2007.map(Cloud20)
NDVI_2007_sn = c2007_sn.qualityMosaic('B11')
NDWI_2007_sn = c2007_sn.qualityMosaic('NDWI')
SAVI_2007_sn = c2007_sn.qualityMosaic('SAVI')
NDGI_2007_sn = c2007_sn.qualityMosaic('NDGI')
NDSI_2007_sn = c2007_sn.qualityMosaic('NDSI')

In [46]:
Map2.addLayer(NDVI_2007_sn, vizParams, 'QMsn de NDVI 2007')
Map2.addLayer(NDWI_2007_sn, vizParams, 'QMsn de NDWI 2007')
Map2.addLayer(SAVI_2007_sn, vizParams, 'QMsn de SAVI 2007')
Map2.addLayer(NDGI_2007_sn, vis_nieve, 'QMsn de NDGI 2007')
Map2.addLayer(NDSI_2007_sn, vis_nieve, 'QMsn de NDSI 2007')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2006

In [47]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l5_2006 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [48]:
coleccion2006 = l5_2006.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [49]:
QM_NDVI_2006 = coleccion2006.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2006 = coleccion2006.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2006 = coleccion2006.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2006 = coleccion2006.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2006 = coleccion2006.qualityMosaic('NDSI') # Normalized difference snow index

In [50]:
Map3.addLayer(QM_NDVI_2006, vizParams, 'QM de NDVI 2006')
Map3.addLayer(QM_NDWI_2006, vizParams, 'QM de NDWI 2006')
Map3.addLayer(QM_SAVI_2006, vizParams, 'QM de SAVI 2006')
Map3.addLayer(QM_NDGI_2006, vis_nieve, 'QM de NDGI 2006')
Map3.addLayer(QM_NDSI_2006, vis_nieve, 'QM de NDSI 2006')
# Map3

In [51]:
c2006_sn = coleccion2006.map(Cloud20)
NDVI_2006_sn = c2006_sn.qualityMosaic('B11')
NDWI_2006_sn = c2006_sn.qualityMosaic('NDWI')
SAVI_2006_sn = c2006_sn.qualityMosaic('SAVI')
NDGI_2006_sn = c2006_sn.qualityMosaic('NDGI')
NDSI_2006_sn = c2006_sn.qualityMosaic('NDSI')

In [52]:
Map4.addLayer(NDVI_2006_sn, vizParams, 'QMsn de NDVI 2006')
Map4.addLayer(NDWI_2006_sn, vizParams, 'QMsn de NDWI 2006')
Map4.addLayer(SAVI_2006_sn, vizParams, 'QMsn de SAVI 2006')
Map4.addLayer(NDGI_2006_sn, vis_nieve, 'QMsn de NDGI 2006')
Map4.addLayer(NDSI_2006_sn, vis_nieve, 'QMsn de NDSI 2006')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2005

In [53]:
start_date = '2004-10-01'
end_date = '2005-03-31'

l5_2005 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [54]:
coleccion2005 = l5_2005.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [55]:
QM_NDVI_2005 = coleccion2005.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2005 = coleccion2005.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2005 = coleccion2005.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2005 = coleccion2005.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2005 = coleccion2005.qualityMosaic('NDSI') # Normalized difference snow index

In [56]:
Map3.addLayer(QM_NDVI_2005, vizParams, 'QM de NDVI 2005')
Map3.addLayer(QM_NDWI_2005, vizParams, 'QM de NDWI 2005')
Map3.addLayer(QM_SAVI_2005, vizParams, 'QM de SAVI 2005')
Map3.addLayer(QM_NDGI_2005, vis_nieve, 'QM de NDGI 2005')
Map3.addLayer(QM_NDSI_2005, vis_nieve, 'QM de NDSI 2005')
# Map3

In [57]:
c2005_sn = coleccion2005.map(Cloud20)
NDVI_2005_sn = c2005_sn.qualityMosaic('B11')
NDWI_2005_sn = c2005_sn.qualityMosaic('NDWI')
SAVI_2005_sn = c2005_sn.qualityMosaic('SAVI')
NDGI_2005_sn = c2005_sn.qualityMosaic('NDGI')
NDSI_2005_sn = c2005_sn.qualityMosaic('NDSI')

In [58]:
Map4.addLayer(NDVI_2005_sn, vizParams, 'QMsn de NDVI 2005')
Map4.addLayer(NDWI_2005_sn, vizParams, 'QMsn de NDWI 2005')
Map4.addLayer(SAVI_2005_sn, vizParams, 'QMsn de SAVI 2005')
Map4.addLayer(NDGI_2005_sn, vis_nieve, 'QMsn de NDGI 2005')
Map4.addLayer(NDSI_2005_sn, vis_nieve, 'QMsn de NDSI 2005')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2009 L7

In [59]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l7_2009 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [60]:
coleccion_l7_2009 = l7_2009.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [61]:
coleccion_l7_2009 = coleccion_l7_2009.set('SENSOR_ID', 'OLI_TIRS')

In [62]:
QM_NDVI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2009_l7 = coleccion_l7_2009.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDSI') # Normalized difference snow index

In [63]:
Map5.addLayer(QM_NDVI_2009_l7, vizParams, 'QM de NDVI 2009_l7')
Map5.addLayer(QM_NDWI_2009_l7, vizParams, 'QM de NDWI 2009_l7')
Map5.addLayer(QM_SAVI_2009_l7, vizParams, 'QM de SAVI 2009_l7')
Map5.addLayer(QM_NDGI_2009_l7, vis_nieve, 'QM de NDGI 2009_l7')
Map5.addLayer(QM_NDSI_2009_l7, vis_nieve, 'QM de NDSI 2009_l7')
# Map5

In [64]:
c2009_l7_sn = coleccion_l7_2009.map(Cloud20)
NDVI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDVI')
NDWI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDWI')
SAVI_2009_l7_sn = c2009_l7_sn.qualityMosaic('SAVI')
NDGI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDGI')
NDSI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDSI')

In [65]:
Map6.addLayer(NDVI_2009_l7_sn, vizParams, 'QMsn de NDVI 2009_l7')
Map6.addLayer(NDWI_2009_l7_sn, vizParams, 'QMsn de NDWI 2009_l7')
Map6.addLayer(SAVI_2009_l7_sn, vizParams, 'QMsn de SAVI 2009_l7')
Map6.addLayer(NDGI_2009_l7_sn, vis_nieve, 'QMsn de NDGI 2009_l7')
Map6.addLayer(NDSI_2009_l7_sn, vis_nieve, 'QMsn de NDSI 2009_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2008 L7

In [66]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l7_2008 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [67]:
coleccion_l7_2008 = l7_2008.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [68]:
coleccion_l7_2008 = coleccion_l7_2008.set('SENSOR_ID', 'OLI_TIRS')

In [69]:
QM_NDVI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008_l7 = coleccion_l7_2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDSI') # Normalized difference snow index

In [70]:
Map5.addLayer(QM_NDVI_2008_l7, vizParams, 'QM de NDVI 2008_l7')
Map5.addLayer(QM_NDWI_2008_l7, vizParams, 'QM de NDWI 2008_l7')
Map5.addLayer(QM_SAVI_2008_l7, vizParams, 'QM de SAVI 2008_l7')
Map5.addLayer(QM_NDGI_2008_l7, vis_nieve, 'QM de NDGI 2008_l7')
Map5.addLayer(QM_NDSI_2008_l7, vis_nieve, 'QM de NDSI 2008_l7')
Map5

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [71]:
c2008_l7_sn = coleccion_l7_2008.map(Cloud20)
NDVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDVI')
NDWI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDWI')
SAVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('SAVI')
NDGI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDGI')
NDSI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDSI')

In [72]:
Map6.addLayer(NDVI_2008_l7_sn, vizParams, 'QMsn de NDVI 2008_l7')
Map6.addLayer(NDWI_2008_l7_sn, vizParams, 'QMsn de NDWI 2008_l7')
Map6.addLayer(SAVI_2008_l7_sn, vizParams, 'QMsn de SAVI 2008_l7')
Map6.addLayer(NDGI_2008_l7_sn, vis_nieve, 'QMsn de NDGI 2008_l7')
Map6.addLayer(NDSI_2008_l7_sn, vis_nieve, 'QMsn de NDSI 2008_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2007 L7

In [73]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l7_2007 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [74]:
coleccion_l7_2007 = l7_2007.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [75]:
QM_NDVI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007_l7 = coleccion_l7_2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDSI') # Normalized difference snow index

In [76]:
Map5.addLayer(QM_NDVI_2007_l7, vizParams, 'QM de NDVI 2007_l7')
Map5.addLayer(QM_NDWI_2007_l7, vizParams, 'QM de NDWI 2007_l7')
Map5.addLayer(QM_SAVI_2007_l7, vizParams, 'QM de SAVI 2007_l7')
Map5.addLayer(QM_NDGI_2007_l7, vis_nieve, 'QM de NDGI 2007_l7')
Map5.addLayer(QM_NDSI_2007_l7, vis_nieve, 'QM de NDSI 2007_l7')
# Map5

In [77]:
c2007_l7_sn = coleccion_l7_2007.map(Cloud20)
NDVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDVI')
NDWI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDWI')
SAVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('SAVI')
NDGI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDGI')
NDSI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDSI')

In [78]:
Map6.addLayer(NDVI_2007_l7_sn, vizParams, 'QMsn de NDVI 2007_l7')
Map6.addLayer(NDWI_2007_l7_sn, vizParams, 'QMsn de NDWI 2007_l7')
Map6.addLayer(SAVI_2007_l7_sn, vizParams, 'QMsn de SAVI 2007_l7')
Map6.addLayer(NDGI_2007_l7_sn, vis_nieve, 'QMsn de NDGI 2007_l7')
Map6.addLayer(NDSI_2007_l7_sn, vis_nieve, 'QMsn de NDSI 2007_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2006 L7

In [79]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l7_2006 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [80]:
coleccion_l7_2006 = l7_2006.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [81]:
QM_NDVI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2006_l7 = coleccion_l7_2006.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDSI') # Normalized difference snow index

In [82]:
Map5.addLayer(QM_NDVI_2006_l7, vizParams, 'QM de NDVI 2006_l7')
Map5.addLayer(QM_NDWI_2006_l7, vizParams, 'QM de NDWI 2006_l7')
Map5.addLayer(QM_SAVI_2006_l7, vizParams, 'QM de SAVI 2006_l7')
Map5.addLayer(QM_NDGI_2006_l7, vis_nieve, 'QM de NDGI 2006_l7')
Map5.addLayer(QM_NDSI_2006_l7, vis_nieve, 'QM de NDSI 2006_l7')
# Map5

In [83]:
c2006_l7_sn = coleccion_l7_2006.map(Cloud20)
NDVI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDVI')
NDWI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDWI')
SAVI_2006_l7_sn = c2006_l7_sn.qualityMosaic('SAVI')
NDGI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDGI')
NDSI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDSI')

In [84]:
Map8.addLayer(NDVI_2006_l7_sn, vizParams, 'QMsn de NDVI 2006_l7')
Map8.addLayer(NDWI_2006_l7_sn, vizParams, 'QMsn de NDWI 2006_l7')
Map8.addLayer(SAVI_2006_l7_sn, vizParams, 'QMsn de SAVI 2006_l7')
Map8.addLayer(NDGI_2006_l7_sn, vis_nieve, 'QMsn de NDGI 2006_l7')
Map8.addLayer(NDSI_2006_l7_sn, vis_nieve, 'QMsn de NDSI 2006_l7')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Indices por colección

### Índices 2008

In [85]:
NDVI_2008 = NDVI_2008_sn.select('B11')
NDWI_2008 = NDWI_2008_sn.select('NDWI') 
SAVI_2008 = SAVI_2008_sn.select('SAVI')
NDGI_2008 = NDGI_2008_sn.select('NDGI')
NDSI_2008 = NDSI_2008_sn.select('NDSI')

In [86]:
Map7.addLayer(NDVI_2008, {'palette': color_terrain}, 'QMsn de NDVI 2008')
Map7.addLayer(NDWI_2008, {'palette': color_ndwi}, 'QMsn de NDWI 2008')
Map7.addLayer(SAVI_2008, {'palette': color_ndvi}, 'QMsn de SAVI 2008')
# Map7.addLayer(NDGI_2008, {'palette': color_glaciar}, 'QMsn de NDGI 2008')
# Map7.addLayer(NDSI_2008, {'palette': color_ndwi}, 'QMsn de NDSI 2008')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2007

In [87]:
NDVI_2007 = NDVI_2007_sn.select('B11')
NDWI_2007 = NDWI_2007_sn.select('NDWI') 
SAVI_2007 = SAVI_2007_sn.select('SAVI')
NDGI_2007 = NDGI_2007_sn.select('NDGI')
NDSI_2007 = NDSI_2007_sn.select('NDSI')

In [88]:
Map7.addLayer(NDVI_2007, {'palette': color_terrain}, 'QMsn de NDVI 2007')
Map7.addLayer(NDWI_2007, {'palette': color_ndwi}, 'QMsn de NDWI 2007')
Map7.addLayer(SAVI_2007, {'palette': color_ndvi}, 'QMsn de SAVI 2007')
# Map7.addLayer(NDGI_2007, {'palette': color_glaciar}, 'QMsn de NDGI 2007')
# Map7.addLayer(NDSI_2007, {'palette': color_ndwi}, 'QMsn de NDSI 2007')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2006

In [89]:
NDVI_2006 = NDVI_2006_sn.select('B11')
NDWI_2006 = NDWI_2006_sn.select('NDWI') 
SAVI_2006 = SAVI_2006_sn.select('SAVI')
NDGI_2006 = NDGI_2006_sn.select('NDGI')
NDSI_2006 = NDSI_2006_sn.select('NDSI')

In [90]:
Map8.addLayer(NDVI_2006, {'palette': color_terrain}, 'QMsn de NDVI 2006')
Map8.addLayer(NDWI_2006, {'palette': color_ndwi}, 'QMsn de NDWI 2006')
Map8.addLayer(SAVI_2006, {'palette': color_ndvi}, 'QMsn de SAVI 2006')
# Map8.addLayer(NDGI_2006, {'palette': color_glaciar}, 'QMsn de NDGI 2006')
# Map8.addLayer(NDSI_2006, {'palette': color_ndwi}, 'QMsn de NDSI 2006')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2005 

In [91]:
NDVI_2005 = NDVI_2005_sn.select('B11')
NDWI_2005 = NDWI_2005_sn.select('NDWI') 
SAVI_2005 = SAVI_2005_sn.select('SAVI')
NDGI_2005 = NDGI_2005_sn.select('NDGI')
NDSI_2005 = NDSI_2005_sn.select('NDSI')

In [92]:
Map8.addLayer(NDVI_2005, {'palette': color_terrain}, 'QMsn de NDVI 2005')
Map8.addLayer(NDWI_2005, {'palette': color_ndwi}, 'QMsn de NDWI 2005')
Map8.addLayer(SAVI_2005, {'palette': color_ndvi}, 'QMsn de SAVI 2005')
# Map8.addLayer(NDGI_2005, {'palette': color_glaciar}, 'QMsn de NDGI 2005')
# Map8.addLayer(NDSI_2005, {'palette': color_ndwi}, 'QMsn de NDSI 2005')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2008 L7

In [93]:
NDVI_2008_l7 = NDVI_2008_l7_sn.select('NDVI')
NDWI_2008_l7 = NDWI_2008_l7_sn.select('NDWI')
SAVI_2008_l7 = SAVI_2008_l7_sn.select('SAVI')
NDGI_2008_l7 = NDGI_2008_l7_sn.select('NDGI')
NDSI_2008_l7 = NDSI_2008_l7_sn.select('NDSI')

In [94]:
Map8.addLayer(NDVI_2008_l7, {'palette': color_terrain}, 'QMsn de NDVI 2008_l7')
Map8.addLayer(NDWI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDWI 2008_l7')
Map8.addLayer(SAVI_2008_l7, {'palette': color_ndvi}, 'QMsn de SAVI 2008_l7')
# Map8.addLayer(NDGI_2008_l7, {'palette': color_glaciar}, 'QMsn de NDGI 2008_l7')
# Map8.addLayer(NDSI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDSI 2008_l7')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Generación de Mosaico

In [95]:
mosaic = ee.ImageCollection([QM_NDGI_2009_l7,NDVI_2008_l7_sn]).mosaic()

In [96]:
NDVI_landsat_7 = mosaic.set('SENSOR_ID', 'ETM+')

In [97]:
cloud_ = ee.Algorithms.Landsat.simpleCloudScore(NDVI_landsat_7)

In [98]:
cloud_ = cloud_.select('cloud')

In [99]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [100]:
resultados= [cloud_, mosaic]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes',
    'QM NDVI']

In [101]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [102]:
def llenado(image):
    return image.where(cloud_.gt(20),NDVI_2009_l7_sn)

In [103]:
img1 = llenado(mosaic)

In [104]:
img1_landsat = img1.set('SENSOR_ID', 'ETM+')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [105]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [106]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [107]:
def llenado2(image):
    return image.where(cloud_img1.gt(20), NDVI_2008_l7_sn)

In [108]:
img2 = llenado2(img1)

In [109]:
img2_landsat = img2.set('SENSOR_ID', 'ETM+')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [110]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [111]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [112]:
def llenado3(image):
    return image.where(cloud_img2.gt(20), NDVI_2007_l7_sn)

In [113]:
img3 = llenado3(img2)

In [114]:
img3_landsat = img3.set('SENSOR_ID', 'ETM+')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [115]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [116]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [117]:
def llenado4(image):
    return image.where(cloud_img3.gt(20), NDVI_2006_l7_sn)

In [118]:
img4 = llenado4(img3)

In [119]:
img4_landsat = img4.set('SENSOR_ID', 'ETM+')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [120]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [121]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [122]:
NDVI_img4 = img4.select('NDVI')
NDWI_img4 = img4.select('NDWI')

In [123]:
map_img4 = folium.Map(location = [-45, -73], zoom_start = 5)
map_img4.add_ee_layer(NDVI_img4, color_terrain, 'IMG4')
display(map_img4)

In [124]:
def llenado5(image):
    return image.where(NDVI_2008_l7.lt(0.25).And(NDWI_2008_l7.gt(0.3)), NDWI_2008_l7_sn)

In [125]:
img5 = llenado5(img4)

In [126]:
resultados= [img4, img5]
labels = [
    'Mosaico 4',
    'Mosaico 5']
vis_params3=  [
     vizParams, 
    vizParams]

In [127]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [128]:
Map10.addLayer(img4, vizParams, 'Mosaico 4')
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…